In [17]:
import os
os.environ["OPENAI_API_KEY"] = ""  # Replace with your actual key

In [61]:
import asyncio
from playwright.async_api import async_playwright

async def run_playwright_async():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://guttikondaparthasai.info/")
        
        # Screenshot
        await page.screenshot(path="example_screenshot.png")
        print("Title:", await page.title())

        await browser.close()

# Run it
await run_playwright_async()


Title: Partha Sai Guttikonda


In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchRun

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Tool
search_tool = DuckDuckGoSearchRun()

# Agent with search capability
tools = [search_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
response = agent.run("What's the latest news about LLaMA 3?")
print(response)


/var/folders/w5/81rp1d0x34nd8g9c4y0lbncc0000gn/T/ipykernel_64374/1883971777.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/var/folders/w5/81rp1d0x34nd8g9c4y0lbncc0000gn/T/ipykernel_64374/1883971777.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(



> Entering new AgentExecutor chain...
I should search for the latest news about LLaMA 3 to provide an accurate answer.
Action: duckduckgo_search
Action Input: LLaMA 3 latest news
Observation: Our new 8B and 70B parameter Llama 3 models are a major leap over Llama 2 and establish a new state-of-the-art for LLM models at those scales. Thanks to improvements in pretraining and post-training, our pretrained and instruction-fine-tuned models are the best models existing today at the 8B and 70B parameter scale. The new Llama 3.3 70B model has just dropped — here's why it's a big deal. Features. ... Get instant access to breaking news, the hottest reviews, great deals and helpful tips. Bringing open intelligence to all, our latest models expand context length to 128K, add support across eight languages, and include Llama 3.1 405B—the first frontier-level open source AI model. Llama 3.1 405B is in a class of its own, with unmatched flexibility, control, and state-of-the-art capabilities that

In [70]:
from langchain.tools import tool
from playwright.async_api import async_playwright
import asyncio

@tool
async def browse_and_screenshot(url: str) -> str:
    """Visit a URL and take a screenshot."""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        screenshot_path = "screenshot.png"
        await page.screenshot(path=screenshot_path)
        await browser.close()
        return f"Screenshot saved to {screenshot_path}"


In [72]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Tools
search = DuckDuckGoSearchRun()
tools = [search, browse_and_screenshot]

# Async agent initialization
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # supports async tools!
    verbose=True
)


In [99]:
async def main():
    result = await agent.arun("Search for the Partha Sai Guttikonda, goto his portfolio page(https://guttikondaparthasai.info/) wait till it loads completely(10 sec) and capture complete page screenshot.")
    print(result)

await main()




> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Partha Sai Guttikonda'}`


Published 5/14/2024 by Jai Prakash Veerla, Partha Sai Guttikonda, Amir Hajighasemi, Jillur Rahman Saurav, Aarti Darji, Cody T. Reynolds, Mohamed Mohamed and 3 more... SpatialVisVR: An Immersive, Multiplexed Medical Image Viewer With Contextual Similar-Patient Search Published 5/9/2024 by Poojitha Thota, Jai Prakash Veerla, Partha Sai Guttikonda, Mohammad S. Nasr, Shirin Nilizadeh, Jacob M. Luber Demonstration of an Adversarial Attack Against a Multimodal Vision Language Model for Pathology Imaging We found 7 Sai Guttikonda's profiles > Get contact information, phone numbers, home addresses, age, background check, photos, and other public records [Updated: Jan 23, 2025]. Partha Sai Guttikonda. SSL auto-renew inside Nginx docker container. About the config crontab inside the Nginx container. Oct 26, 2021. 1. Oct 26, 2021. 1. Gökhan Şengün. SSL Sertifika Tipleri ... Sai Balaj

# Full Flow

In [117]:
# -----------------------------
# 📦 Step 1: Imports & Setup
# -----------------------------
import os
import json
import re
import requests
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [119]:
# 🧠 LLM setup
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# -----------------------------
# 🧠 Step 2: Prompt Template
# -----------------------------
mcp_prompt = PromptTemplate.from_template("""
You're a browser automation assistant. Convert the following task into a **pure JSON array** of Playwright MCP `actions`.

TASK: {task}

RESPONSE FORMAT (must match):
[
  {{ "type": "goto", "url": "..." }},
  {{ "type": "wait", "time": ... }},
  ...
]
Return only the JSON. No markdown. No explanation.
""")

mcp_chain = LLMChain(llm=llm, prompt=mcp_prompt)

In [121]:
# -----------------------------
# 💬 Step 3: User Task
# -----------------------------
task_prompt = (
    "Go to Partha Sai Guttikonda's portfolio page at https://guttikondaparthasai.info/, "
    "wait for 10 seconds, and take a full-page screenshot named 'partha_mcp_screenshot.png'."
)

generated = mcp_chain.run(task=task_prompt)

In [123]:
# -----------------------------
# 🧼 Step 4: Extract JSON from LLM Output
# -----------------------------
match = re.search(r"\[\s*{.*?}\s*\]", generated, re.DOTALL)
if match:
    cleaned_json = match.group(0)
    try:
        actions = json.loads(cleaned_json)
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", e)
        print("Original Output:\n", generated)
        raise
else:
    print("❌ Failed to parse JSON. Full LLM output:\n", generated)
    raise ValueError("No valid JSON array found.")

In [125]:
# -----------------------------
# 🌐 Step 5: Send to MCP Server
# -----------------------------
MCP_URL = "http://localhost:8931/api/command"  # make sure MCP is running

payload = {"actions": actions}

try:
    res = requests.post(MCP_URL, json=payload)
    print("✅ MCP Server Response:")
    print(res.json())
except Exception as e:
    print("❌ MCP Server Error:", e)

✅ MCP Server Response:
❌ MCP Server Error: Expecting value: line 1 column 1 (char 0)


In [127]:
payload

{'actions': [{'type': 'goto', 'url': 'https://guttikondaparthasai.info/'},
  {'type': 'wait', 'time': 10000},
  {'type': 'screenshot', 'path': 'partha_mcp_screenshot.png'}]}